In [1]:
import crocoddyl
import pinocchio
import numpy as np

class DifferentialActionModelCartpole(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        crocoddyl.DifferentialActionModelAbstract.__init__(self, crocoddyl.StateVector(4), 1, 6) # nu = 1; nr = 6
        self.unone = np.zeros(self.nu)

        self.m1 = 1.
        self.m2 = .1
        self.l  = .5
        self.g  = 9.81
        self.costWeights = [ 1., 1., 0.1, 0.001, 0.001, 1. ]  # sin, 1-cos, x, xdot, thdot, f
        
    def calc(self, data, x, u=None):
        if u is None: u=model.unone
        # Getting the state and control variables
        x, th, xdot, thdot = np.asscalar(x[0]), np.asscalar(x[1]), np.asscalar(x[2]), np.asscalar(x[3])
        f = np.asscalar(u[0])

        # Shortname for system parameters
        m1, m2, l, g = self.m1, self.m2, self.l, self.g
        s, c = np.sin(th), np.cos(th)

        # Defining the equation of motions
        m = m1 + m2
        mu = m1 + m2 * s**2
        xddot  = (f     + m2 * c * s * g - m2 * l * s * thdot**2 ) / mu
        thddot = (c * f / l + m * g * s / l  - m2 * c * s * thdot**2 ) / mu
        data.xout = np.matrix([ xddot,thddot ]).T

        # Computing the cost residual and value
        data.r = np.matrix(self.costWeights * np.array([ s, 1-c, x, xdot, thdot, f ])).T
        data.cost = .5* np.asscalar(sum(np.asarray(data.r)**2))

    def calcDiff(self,data,x,u=None,recalc=True):
        # Advance user might implement the derivatives
        pass

In [2]:
 cartpoleDAM = DifferentialActionModelCartpole()
cartpoleData = cartpoleDAM.createData()
x = cartpoleDAM.state.rand()
u = np.zeros(1)
cartpoleDAM.calc(cartpoleData,x,u)

/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


In [3]:
cartpoleND = crocoddyl.DifferentialActionModelNumDiff(cartpoleDAM, True) 

In [28]:
 timeStep = 5e-2
cartpoleIAM = crocoddyl.IntegratedActionModelEuler(cartpoleND, timeStep)

 ###########################################################################
################## TODO: Tune the weights for each cost ###################
###########################################################################
terminalCartpole = DifferentialActionModelCartpole()
terminalCartpoleDAM = crocoddyl.DifferentialActionModelNumDiff(terminalCartpole, True)
terminalCartpoleIAM = crocoddyl.IntegratedActionModelEuler(terminalCartpoleDAM)

terminalCartpole.costWeights[0] = 1
terminalCartpole.costWeights[1] = 100
terminalCartpole.costWeights[2] = 1.
terminalCartpole.costWeights[3] = 0.1
terminalCartpole.costWeights[4] = 0.01
terminalCartpole.costWeights[5] = 0.0001

In [29]:
# Fill the number of knots (T) and the time step (dt)
x0 = np.matrix([ 0.2, 3.14, 0.2, 0.1 ]).T
T  = 50
problem = crocoddyl.ShootingProblem(x0, [ cartpoleIAM ]*T, cartpoleIAM)

In [30]:
# Creating the DDP solver
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

In [31]:
# Solving this problem
done = ddp.solve([], [], 1000)
print(done)
print(ddp.us)

True
[matrix([[-0.0297793]]), matrix([[-0.02918955]]), matrix([[-0.02784566]]), matrix([[-0.02581924]]), matrix([[-0.02321744]]), matrix([[-0.02017566]]), matrix([[-0.0168489]]), matrix([[-0.01340239]]), matrix([[-0.01000196]]), matrix([[-0.00680492]]), matrix([[-0.00395164]]), matrix([[-0.00155838]]), matrix([[0.00028829]]), matrix([[0.00153536]]), matrix([[0.00216503]]), matrix([[0.00219458]]), matrix([[0.00167383]]), matrix([[0.00068093]]), matrix([[-0.00068364]]), matrix([[-0.00230373]]), matrix([[-0.0040554]]), matrix([[-0.00581419]]), matrix([[-0.00746263]]), matrix([[-0.00889637]]), matrix([[-0.01002984]]), matrix([[-0.01080015]]), matrix([[-0.01116968]]), matrix([[-0.01112702]]), matrix([[-0.01068653]]), matrix([[-0.00988579]]), matrix([[-0.00878216]]), matrix([[-0.00744792]]), matrix([[-0.00596479]]), matrix([[-0.00441819]]), matrix([[-0.00289146]]), matrix([[-0.00146075]]), matrix([[-0.00019057]]), matrix([[0.00086955]]), matrix([[0.0016873]]), matrix([[0.00224866]]), matrix(

/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/home/a/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


In [32]:
xs = ddp.xs

In [33]:
%%capture
%matplotlib inline
from cartpole_utils import animateCartpole
anim = animateCartpole(xs)
from IPython.display import HTML
# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

In [27]:
HTML(anim.to_html5_video())

In [34]:
xs

[matrix([[0.2 ],
         [3.14],
         [0.2 ],
         [0.1 ]]), matrix([[0.20992164],
         [3.14523483],
         [0.19843288],
         [0.10469664]]), matrix([[0.21977925],
         [3.15041909],
         [0.19715215],
         [0.10368511]]), matrix([[0.2295889 ],
         [3.15526632],
         [0.19619302],
         [0.09694466]]), matrix([[0.23936755],
         [3.15950488],
         [0.195573  ],
         [0.08477114]]), matrix([[0.2491321 ],
         [3.1628931 ],
         [0.19529087],
         [0.06776436]]), matrix([[0.25889844],
         [3.16523301],
         [0.1953268 ],
         [0.04679837]]), matrix([[0.26868062],
         [3.16638181],
         [0.19564359],
         [0.02297592]]), matrix([[ 2.78490060e-01],
         [ 3.16626031e+00],
         [ 1.96188884e-01],
         [-2.42999573e-03]]), matrix([[ 0.28833497],
         [ 3.16485807],
         [ 0.1968982 ],
         [-0.02804471]]), matrix([[ 0.29821991],
         [ 3.16223474],
         [ 0.19769871]

In [36]:
us = ddp.us

In [37]:
us

[matrix([[-0.0297793]]),
 matrix([[-0.02918955]]),
 matrix([[-0.02784566]]),
 matrix([[-0.02581924]]),
 matrix([[-0.02321744]]),
 matrix([[-0.02017566]]),
 matrix([[-0.0168489]]),
 matrix([[-0.01340239]]),
 matrix([[-0.01000196]]),
 matrix([[-0.00680492]]),
 matrix([[-0.00395164]]),
 matrix([[-0.00155838]]),
 matrix([[0.00028829]]),
 matrix([[0.00153536]]),
 matrix([[0.00216503]]),
 matrix([[0.00219458]]),
 matrix([[0.00167383]]),
 matrix([[0.00068093]]),
 matrix([[-0.00068364]]),
 matrix([[-0.00230373]]),
 matrix([[-0.0040554]]),
 matrix([[-0.00581419]]),
 matrix([[-0.00746263]]),
 matrix([[-0.00889637]]),
 matrix([[-0.01002984]]),
 matrix([[-0.01080015]]),
 matrix([[-0.01116968]]),
 matrix([[-0.01112702]]),
 matrix([[-0.01068653]]),
 matrix([[-0.00988579]]),
 matrix([[-0.00878216]]),
 matrix([[-0.00744792]]),
 matrix([[-0.00596479]]),
 matrix([[-0.00441819]]),
 matrix([[-0.00289146]]),
 matrix([[-0.00146075]]),
 matrix([[-0.00019057]]),
 matrix([[0.00086955]]),
 matrix([[0.0016873]])